In [1]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 63 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 122 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 25.6 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 8.2 MB/s  eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import json

def fetch_trials(diseases, keywords):
    base_url = "https://clinicaltrials.gov/api/query/full_studies"
    results = []

    for disease in diseases:
        for keyword in keywords:
            params = {
                'expr': f'{disease} AND {keyword}',
                'min_rnk': 1,
                'max_rnk': 50,
                'fmt': 'json'
            }
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                trials = data.get('FullStudiesResponse', {}).get('FullStudies', [])
                for trial in trials:
                    #if trial_has_results(trial):
                        trial_info = process_trial_data(trial)
                        results.append(trial_info)
            else:
                print(f"Failed to fetch data for {disease} with keyword '{keyword}'. Status code: {response.status_code}")

    return results

def trial_has_results(trial):
    status_module = trial['Study']['ProtocolSection']['StatusModule']
    return 'ResultsFirstPosted' in status_module

def process_trial_data(trial):
    study = trial['Study']
    protocol_section = study['ProtocolSection']
    identification_module = protocol_section['IdentificationModule']
    status_module = protocol_section['StatusModule']
    design_module = protocol_section['DesignModule']

    trial_info = {
        'NCT Number': identification_module.get('NCTId', ''),
        'Title': identification_module.get('OfficialTitle', identification_module.get('BriefTitle', '')),
        'Status': status_module.get('OverallStatus', ''),
        'Study Type': identification_module.get('StudyType', ''),
        'Enrollment': status_module.get('EnrollmentCount', ''),
        'Conditions': design_module.get('Conditions', {}).get('Condition', []),
        'Keywords': design_module.get('Keywords', {}).get('Keyword', []),
        #'Animal Details': extract_animal_details(design_module)
    }

    return trial_info

def extract_animal_details(design_module):
    # Placeholder for actual data extraction logic, depending on the available details in the API response
    # This might need to be adapted based on the specific structure and content of the study descriptions
    animal_details = {
        'Animal Model': 'Details about the animal model used',
        'Age': 'Animal age',
        'Weight': 'Animal weight',
        'Sex': 'Animal sex',
        'Disease Model': 'Description of the disease model',
        'Endpoint': 'Study endpoint details'
    }
    return animal_details

# Diseases and keywords to search
diseases = ['IBS', "Crohn's Disease", 'Celiac Disease'] #, "Crohn's Disease", 'Celiac Disease'
keywords = ['preclinical', 'rat', 'rodent', 'mice', 'animal', 'pig', 'hamster'] #, 'rat', 'rodent', 'mice', 'animal', 'pig', 'hamster'

# Fetch and print the trial data
trial_data = fetch_trials(diseases, keywords)
for data in trial_data:
    print(json.dumps(data, indent=4))


{
    "NCT Number": "NCT05249023",
    "Title": "Mode of Action of Butyrate in the Human Colon",
    "Status": "Completed",
    "Study Type": "",
    "Enrollment": "",
    "Conditions": [],
    "Keywords": []
}
{
    "NCT Number": "NCT03003260",
    "Title": "A Randomized, Double-blind, Cross-over Trial of the Effect of CanChew\u00ae Cannabidiol (CBD) Containing Chewing Gum on Patients With Irritable Bowel Syndrome",
    "Status": "Unknown status",
    "Study Type": "",
    "Enrollment": "",
    "Conditions": [],
    "Keywords": []
}
{
    "NCT Number": "NCT00461682",
    "Title": "A Double-blind, Randomised, Placebo Controlled, Single Dose, Two-period Crossover Study to Investigate the Therapeutic Potential of the TRPV1 Antagonist SB-705498 in Treatment of Subjects With Rectal Hypersensitivity Including Irritable Bowel Syndrome.",
    "Status": "Terminated",
    "Study Type": "",
    "Enrollment": "",
    "Conditions": [],
    "Keywords": []
}
{
    "NCT Number": "NCT04005456",
    "T